# **CPNS 2019** 

In [1]:
# import library
import requests
import json 
import pandas as pd
import numpy as np
import time

### **User Agent**

In [2]:
UA= "Mozilla/5.0 (Macintosh; InterruptedErrorel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
headers = {"User-Agent": UA}

### **URL SSCN BKN**

In [3]:
url = "https://sscndata.bkn.go.id/spf" # method post
parameters = '{"draw":1,"columns":[{"data":"insNm","name":"","searchable":true,"orderable":false,"search":{"value":"","regex":false}},{"data":"jabNm","name":"","searchable":true,"orderable":false,"search":{"value":"","regex":false}},{"data":"lokasiNm","name":"","searchable":true,"orderable":false,"search":{"value":"","regex":false}},{"data":"pendidikanNm","name":"","searchable":true,"orderable":false,"search":{"value":"","regex":false}},{"data":"jenisFormasiNm","name":"","searchable":true,"orderable":false,"search":{"value":"","regex":false}},{"data":"jumlahFormasi","name":"","searchable":false,"orderable":false,"search":{"value":"","regex":false}},{"data":"jumlahSubmit","name":"","searchable":false,"orderable":false,"search":{"value":"","regex":false}}],"order":[{"column":0,"dir":"asc"}],"start":0,"length":10,"search":{"value":"","regex":false}}'

# parameters for fetch the data
def params_rows(draw=1, start=0, rows=200):
    data_json = json.loads(parameters)
    data_json['draw']=draw
    data_json['start']=start
    data_json['length']=rows
    return data_json

### **Get the Information about Total Rows**

In [4]:
# set parameters
params = params_rows(draw=1, start=0, rows=1)

# get the total rows
rs = requests.post(url, json=params, headers=headers)
rs_total = json.loads(rs.text)['recordsTotal']
print(rs_total)

# set range for looping
rs_rows = 10000
range_loop = np.ceil(rs_total/rs_rows).astype(int)
print(range_loop)

108209
11


### **Fetch the data**

In [5]:
cols = ['insKd','insNm','jabNm','jenisFormasiNm','jumlahFormasi','jumlahSubmit','lokasiNm','pendidikanNm']
df_final = pd.DataFrame(columns=cols) 
j=0
for i in range(range_loop):
    params = params_rows(draw=1, start=j, rows=rs_rows)
    j = j+rs_rows # will use for start row in every url
    rs = requests.post(url, json=params)
    rs = json.loads(rs.text)['data']
    result = pd.DataFrame(eval(json.dumps(rs)))
    print(result.shape)
    df_final = pd.concat([df_final,result],ignore_index=True).drop_duplicates().reset_index(drop=True)
    time.sleep(5)

(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(10000, 8)
(8209, 8)


In [6]:
df_final.head(5)

,insKd,insNm,jabNm,jenisFormasiNm,jumlahFormasi,jumlahSubmit,lokasiNm,pendidikanNm
0,5473,Pemerintah Kota Sawahlunto,PELAKSANA/TERAMPIL - ASISTEN APOTEKER,PENYANDANG DISABILITAS,1,0,"RUMAH SAKIT UMUM DAERAH (RSUD), SEKSI PENUNJAN...",D-III FARMASI
1,3013,Kementerian Ketenagakerjaan,AHLI PERTAMA - ARSIPARIS,UMUM,2,229,"SEKRETARIAT JENDERAL, BIRO PERENCANAAN DAN MAN...",S-1 ADMINISTRASI / S-1 ADMINISTRASI PERKANTORA...
2,3013,Kementerian Ketenagakerjaan,ANALIS BAHAN PENYELENGGARAAN PROGRAM PEMAGANGAN,UMUM,1,99,DIREKTORAT JENDERAL PEMBINAAN PELATIHAN DAN PR...,S-1 ADMINISTRASI / S-1 HUBUNGAN INTERNASIONAL ...
3,3013,Kementerian Ketenagakerjaan,PELAKSANA/TERAMPIL - ARSIPARIS,UMUM,1,42,DIREKTORAT JENDERAL PEMBINAAN PELATIHAN DAN PR...,D-III ADMINISTRASI / D-III ADMINISTRASI PERKAN...
4,3013,Kementerian Ketenagakerjaan,ANALIS ADVOKASI HUKUM,UMUM,1,26,"SEKRETARIAT JENDERAL, BIRO HUKUM",S-1 HUKUM


In [7]:
df_final.to_csv("cpns_2019.csv", encoding='utf-8',index=False)